sort 라이브러리 링크: https://github.com/abewley/sort

SIMPLE ONLINEANDREALTIMETRACKING(SORT)
-	바운딩 박스의 위치와 크기만을 사용해 모션 예측과 데이터 연결을 수행
-	Kalman filter를 통한 모션 예측
-	헝가리안 알고리즘을 이용한 데이터 연결
-	트랙 생성 및 삭제 전략을 사용하여 다중 객체 추적을 간소화

구조
-	검출(Detection): Faster R-CNN을 사용하여 객체를 검출하고, 결과 중 ‘사람’에 해당하는 객체만 필터링
-	추정 모델(Estimation Model): 칼만 필터를 활용하여 객체의 위치, 속도 등을 추정. 객체의 위치는 중심 좌표와 크기를 통해 표현
-	데이터 연결(Data Association): 프레임 간 객체를 연결하기 위해 헝가리안 알고리즘을 사용. 검출된 객체와 기존 추적 객체 간의 거리 비용을 계산하여 최적의 매칭을 찾고, IOU(Intersection Over Union)을 사용하여 탐지된 객체와 추적 객체 간의 유사도를 측정
-	트랙의 생성 및 삭제: 새로운 객체는 일정 기준에 따라 새로운 트랙으로 생성되고, 일정 시간 동안 검출되지 않은 객체는 삭제. 이를 통해 추적 트랙 수가 과도하게 증가하는 것을 방지

SORT Class
- 객체 추적을 관리하는 핵심 클래스로, 각 프레임마다 탐지된 객체의 위치를 기반으로 기존에 추적 중이던 객체의 상태를 갱신하고, 새롭게 등장한 객체에는 새로운 ID를 부여하는 역할
- 일정 시간 동안 탐지되지 않은 객체는 추적 대상에서 제거

- max_age: 추적 중인 객체가 탐지되지 않더라도 추적을 유지할 최대 프레임 수. 일정 시간 동안 객체가 탐지되지 않으면, 이 값에 따라 해당 객체의 추적을 중단
- min_hits: 추적할 객체가 되기 위해 필요한 최소 탐지 횟수. 객체가 한 번 탐지되었다고 바로 추적 대상으로 삼지 않고, 여러 번 탐지된 후에야 유효한 객체로 간주하여 추적을 시작
- iou_threshold: 탐지된 객체와 기존 추적 객체 간의 매칭 시 사용되는 IOU 임계값으로, 이 값 이상이어야 두 객체를 동일한 객체로 매칭
- update 메서드: 프레임마다 호출되며, 탐지된 객체와 기존 추적 객체를 매칭하고 각 객체의 상태를 갱신하는 역할

1. 프레임마다 객체의 위치 예측
 - 현재 추적 중인 모든 객체의 다음 위치를 예측

2. 탐지된 객체와 추적 중인 객체 매칭
 - associate_detections_to_trackers 함수를 호출하여 탐지된 객체와 기존 추적 객체 간의 최적 매칭을 수행

3. 매칭된 객체 갱신
 - 매칭된 객체들은 KalmanBoxTracker의 update 메서드를 통해 상태 벡터를 갱신
 - 새로운 탐지 정보가 반영되어 객체의 위치가 보정

4. 매칭되지 않은 탐지 객체에 대한 새로운 추적기 생성
 - 새로 탐지된 객체들 중 기존 객체와 매칭되지 않은 객체들은 새로운 객체로 간주되어 KalmanBoxTracker 인스턴스로 생성
 - trackers 리스트에 추가되어 이후 프레임에서 추적

5. 추적기 상태 업데이트 및 만료된 추적기 제거
 - 각 추적기의 상태를 업데이트하고, 일정 시간 동안 탐지되지 않은 객체는 제거

6. 반환
 - 모든 추적 객체의 상태를 ret에 저장한 후, 이를 반환
 - ret에는 [x1, y1, x2, y2, ID] 형식의 정보가 포함되며, 이 정보는 현재 프레임에서 추적 중인 모든 객체의 바운딩 박스와 ID를 나타냄

In [ ]:
class Sort(object):
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0, 5))):
        self.frame_count += 1
        trks = np.zeros((len(self.trackers), 5))
        to_del = []
        ret = []

        for t, trk in enumerate(trks):
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets, trks, self.iou_threshold)

        for m in matched:
            self.trackers[m[1]].update(dets[m[0], :])

        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i, :])
            self.trackers.append(trk)

        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]
            if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d, [trk.id + 1])).reshape(1, -1))  # MOT 벤치마크는 ID가 양수여야 함
            i -= 1
            if trk.time_since_update > self.max_age:
                self.trackers.pop(i)

        if len(ret) > 0:
            return np.concatenate(ret)
        return np.empty((0, 5))


Kalman filter
- 시스템 동역학 모델(이론)과 측정값의 노이즈를 최대한 제거(필터링)하면서, 불완전한 모델 예측 값과 불완전한 측정 값 사이의 최적의 시스템 상태를 추정하는 알고리즘
- 프로세스 노이즈와 관측 노이즈가 모두 평균이 0인 정규분포(zero mean Gaussian distribution)를 따른다고 가정함.
- 이전 시간(n)과 현재 시간(n+1)에 대해 계산
- 실시간 시스템, 임베디드 시스템에 적합
- 최적의 추정 값을 제공 및 추정 오차를 최소화


칼만 필터 추적기
- 각 객체의 위치와 크기를 예측하고 업데이트하며, 이를 통해 객체가 프레임 사이에서 이동할 때 지속적으로 추적할 수 있게 함.

칼만 필터 구성 요소
1. 상태 벡터(x)
 - 객체의 상태를 나타내는 벡터로 위치(x, y), 크기(s), 가로세로 비율(r), 속도 정보를 포함
 - 상태 벡터는 7차원 벡터로 구성: [x, y, s, r, ẋ, ẏ, ṡ].
 - ẋ, ẏ, ṡ는 각각 x, y 위치와 스케일에 대한 속도 성분

2. 상태 전이 행렬(F)
 - 객체가 이동하는 과정에서 현재 상태 벡터를 다음 상태 벡터로 변환하는 역할
 - self.kf.F로 정의되며, 상수 속도 모델을 가정하여 설계

3. 관측 모델 행렬(H)
 - 바운딩 박스 위치 정보를 칼만 필터의 상태 벡터와 비교할 수 있도록 하는 행렬
 - 바운딩 박스의 위치 정보([x, y, s, r])만 관측되므로, 관측 모델 행렬은 self.kf.H로 설정

4. 상태 공분산 행렬(P)
 - 추적 상태에 대한 불확실성을 나타내며, 칼만 필터가 각 상태 변수를 얼마나 신뢰하는지를 나타냄
 - 초기 속도에 대한 불확실성을 높게 설정하여 초기 상태 공분산 행렬을 정의

5. 프로세스 잡음 공분산 행렬(Q) 및 관측 잡음 공분산 행렬(R)
 - Q: 추적 상태가 시간에 따라 변할 때 발생하는 잡음
 - R: 관측 값의 잡음
 -필터링의 부드러움을 조정하는 데 사용


칼만 필터 동작 과정
1. 예측 단계(predict)
 - F 행렬을 이용해 이전 상태 벡터에서 현재 프레임의 상태를 예측
 - 예측된 상태는 Q 행렬을 통해 잡음을 반영

2. 업데이트 단계 (update)
 - 탐지된 객체가 관측되면, H 행렬을 사용해 관측 벡터와 상태 벡터를 비교하고, R 행렬을 통해 잡음을 고려하여 상태를 갱신
 - 상태 벡터는 관측 정보에 의해 보정되고, 상태 공분산 행렬도 갱신

In [ ]:
class KalmanBoxTracker(object):
  count = 0

  def __init__(self, bbox):

    self.kf = KalmanFilter(dim_x=7, dim_z=4)
    self.kf.F = np.array([[1,0,0,0,1,0,0],
                          [0,1,0,0,0,1,0],
                          [0,0,1,0,0,0,1],
                          [0,0,0,1,0,0,0],
                          [0,0,0,0,1,0,0],
                          [0,0,0,0,0,1,0],
                          [0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],
                          [0,1,0,0,0,0,0],
                          [0,0,1,0,0,0,0],
                          [0,0,0,1,0,0,0]])


    self.kf.R[2:, 2:] *= 10.
    self.kf.P[4:, 4:] *= 1000.
    self.kf.P *= 10.
    self.kf.Q[-1, -1] *= 0.01
    self.kf.Q[4:, 4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0


Hungarian Algorithm
- 1대1 대응 매칭 문제를 해결하는 최적화 알고리즘
- 할당, 최적 매칭 문제에 적용
- 응용 분야: 작업 할당 문제, 다중 객체 추적에서의 데이터 연결(Data Association)
- 다중 객체 추적: 추적 대상 객체와 프레임마다 검출된 객체 위치를 매칭할 때 사용. 추적 대상과 검출 위치 간의 비용(거리)을 최소화하며 매칭함으로써 객체를 정확하게 연결


헝가리안 알고리즘을 이용한 선형 할당
- 객체 추적에서 각 프레임마다 새로 탐지된 객체와 이전 프레임에서 추적 중인 객체 간에 최적의 매칭을 찾아내는데 사용
- cost_matrix: 비용 행렬로, 새로 탐지된 객체와 기존에 추적 중인 객체 간의 유사도나 거리(IOU 거리)가 포함됨 행렬. 이 행렬의 각 요소는 두 객체가 매칭될 때의 비용을 나타냄
- 헝가리안 알고리즘 적용: cost_matrix가 주어지면, 헝가리안 알고리즘을 통해 비용이 최소가 되는 매칭을 찾아냄
- linear_sum_assignment: 비용 행렬을 최소화 할 수 있는 행렬의 행과 열 인덱스를 반환
- 결과: zip(x, y)를 통해 두 인덱스를 묶어 매칭 결과를 반환(x, y: 각각 최적 매칭을 이루는 탐지된 객체의 인덱스와 기존 추적 객체의 인덱스)

헝가리안 알고리즘의 동작
1. 비용행렬 생성:
 - cost_matrix는 탐지된 객체와 추적 중인 객체 간의 매칭 비용. iou_matrix를 음수로 변환하여 두 객체가 최대한 유사한 경우 최소 비용이 되도록 함.

2. 헝가리안 알고리즘 적용:
 - linear_sum_assignment 함수는 이 비용 행렬을 기반으로 최적 매칭을 찾음. 이를 통해 각 탐지된 객체가 가장 유사한 기존 추적 객체와 매칭되며, 매칭되지 않은 객체는 새 객체로 간주하거나, 일정 시간 동안 추적되지 않으면 삭제

3. 출력:
 - 최적의 매칭 인덱스를 반환하며, 이 인덱스를 이용해 탐지된 객체와 추적 중인 객체를 연결

In [ ]:
def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


탐지된 객체와 추적 객체 연결(associate_detections_to_trackers)
- 탐지된 객체(즉, 현재 프레임에서 새로 발견된 객체)와 추적 중인 객체(이전 프레임에서부터 계속 추적 중인 객체) 간의 최적의 매칭을 찾아 연결
- IOU(Intersection over Union) 값에 기반한 매칭을 수행하여, 탐지된 객체가 추적 객체와 동일한 객체임을 판단
- ID 전환을 최소화하고 지속적인 객체 추적을 가능하게함

함수 실행 단계
1. 입력
 - detections: 현재 프레임에서 탐지된 객체의 바운딩 박스 리스트
 - trackers: 이전 프레임에서 추적 중이던 객체의 바운딩 박스 리스트
 - iou_threshold: 매칭을 결정할 때 사용할 IOU의 임계값. 이 값 이상일 때만 두 객체를 동일 객체로 간주

2. IOU 거리 행렬 생성
 - iou_batch 함수를 호출하여 탐지된 객체와 추적 객체 간의 IOU(Intersection over Union) 값을 계산하고, IOU 거리 행렬을 생성
 - iou_matrix는 각 탐지된 객체와 추적 중인 객체 간의 IOU 값을 나타내며, 이 값이 높을수록 두 객체가 동일 객체일 가능성이 높음

 3. 매칭 수행
  - 임계값 기반 매칭: 먼저, IOU 값이 iou_threshold를 초과하는 객체들만 고려하여 일대일 대응을 구성
  - 헝가리안 알고리즘 기반 매칭: 임계값을 넘는 매칭이 불완전할 경우, linear_assignment 함수를 사용하여 헝가리안 알고리즘을 통해 비용이 최소화되는 최적 매칭을 수행
  - linear_assignment 함수는 비용 행렬로 -iou_matrix를 사용하여 IOU 값이 가장 높은 쌍이 최적 매칭이 되도록 설정

4. 매칭되지 않은 탐지 객체와 추적 객체 식별
 - 매칭되지 않은 객체를 찾기 위해 matched_indices에 포함되지 않은 탐지 및 추적 객체의 인덱스를 각각 unmatched_detections와 unmatched_trackers에 추가
 - 매칭되지 않은 탐지 객체는 새로운 객체로 간주
 - 매칭되지 않은 추적 객체는 일정 시간이 지나면 추적에서 제외

5. 낮은 IOU로 매칭된 것 제거
 - matches 리스트에는 IOU 임계값을 초과한 매칭만 추가

6. 출력
 - matches: 매칭된 탐지 객체와 추적 객체의 인덱스 쌍으로 이루어진 리스트
 - unmatched_detections: 매칭되지 않은 탐지 객체의 인덱스 리스트
 - unmatched_trackers: 매칭되지 않은 추적 객체의 인덱스 리스트

In [ ]:
def associate_detections_to_trackers(detections, trackers, iou_threshold=0.3):

    if len(trackers) == 0:
        return np.empty((0, 2), dtype=int), np.arange(len(detections)), np.empty((0, 5), dtype=int)

    iou_matrix = iou_batch(detections, trackers)

    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))

    unmatched_detections = []
    for d, det in enumerate(detections):
        if d not in matched_indices[:, 0]:
            unmatched_detections.append(d)
    unmatched_trackers = []
    for t, trk in enumerate(trackers):
        if t not in matched_indices[:, 1]:
            unmatched_trackers.append(t)

    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1, 2))
    if len(matches) == 0:
        matches = np.empty((0, 2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)
